In [1]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from MyLLM.CrispyLLM_RoPE2.modeling_crispy_rope import CrispyLLMConfig, CrispyForCausalLM
from transformers import XLMRobertaTokenizer, PreTrainedTokenizerFast
from datasets import load_dataset, concatenate_datasets
"""
# 3. Kayıt (Auto ile kullanabilmek için)
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)"""  

'\n# 3. Kayıt (Auto ile kullanabilmek için)\nAutoConfig.register("crispy", CrispyLLMConfig)\nAutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)'

In [3]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"

In [4]:
max_seq_length = 1024*2  # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. bfloat16 for Tesla T4, V100, bfloat16 for Ampere+
load_in_4bit = False 
load_in_8bit = False 

In [5]:
# XLM-Roberta tokenizer yükleniyor
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
#tokenizer.model_max_length = max_seq_length*8  # Örneğin 4096 yapmak


In [6]:
special_tokens_dict = {
    "bos_token": "<s>",
    "eos_token": "<|eot_id|>",
    "additional_special_tokens":  [
        "<|im_start|>", "<|im_end|>",
        "<|system|>", "<|user|>", "<|assistant|>",
        "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"
    ]
}


#tokenizer.add_special_tokens(special_tokens_dict)

In [7]:



#crispy_config = CrispyLLMConfig(attn_implementation="flash_attention_2", use_flash_attention_2=True, vocab_size=len(tokenizer.get_vocab()), n_heads=16, max_seq_len=max_seq_length, hidden_size=64*16, num_hidden_layers=16, dtype="bfloat16")

#crispy_config._attn_implementation_autoset = True  # 👈 Buraya ekliyorsun

#model = AutoModelForCausalLM.from_config(crispy_config)

In [8]:
model_path = "hosmankarabulut/Crispy-2.8B-CLM"

model = AutoModelForCausalLM.from_pretrained(model_path,  
                                            attn_implementation="flash_attention_2",
                                            trust_remote_code=True,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto"
      ) 
tokenizer = AutoTokenizer.from_pretrained(model_path)


📦 2 adet safetensors dosyası bulundu. Yükleniyor...
   ↪️ hosmankarabulut/Crispy-2.8B-CLM/model-00001-of-00002.safetensors
   ↪️ hosmankarabulut/Crispy-2.8B-CLM/model-00002-of-00002.safetensors


In [9]:
# Tokenizer’a yeni token eklediysen bunu yapman gerekir
#model.resize_token_embeddings(len(tokenizer))

In [10]:
# Tokenizer'dan alınan vocab size
tokenizer_vocab_size = len(tokenizer.get_vocab())
print(f"🔤 Tokenizer vocab size: {tokenizer_vocab_size}")

# Modelin token embedding katmanından alınan vocab size
model_embedding_vocab_size = model.embedding.token_embedding.embedding_layer.num_embeddings
print(f"🧠 Model token embedding vocab size: {model_embedding_vocab_size}")

# Modelin lm_head katmanından alınan çıkış boyutu
model_lm_head_vocab_size = model.lm_head.out_features
print(f"🎯 Model lm_head vocab size: {model_lm_head_vocab_size}")

# Hepsi eşleşiyor mu?
if tokenizer_vocab_size == model_embedding_vocab_size == model_lm_head_vocab_size:
    print("✅ Tokenizer ve model vocab boyutları uyumlu.")
else:
    print("⚠️ UYARI: Vocab size değerleri eşleşmiyor!")
    model.resize_token_embeddings(len(tokenizer))
    print(f"Model tokenizera göre ayarlandı {model_lm_head_vocab_size} --> {tokenizer_vocab_size}")


🔤 Tokenizer vocab size: 50000
🧠 Model token embedding vocab size: 50000
🎯 Model lm_head vocab size: 50000
✅ Tokenizer ve model vocab boyutları uyumlu.


In [11]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(50000, 1920)
    )
  )
  (decoderBlocks): ModuleList(
    (0-29): 30 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1920, out_features=5760, bias=True)
        (o_proj): Linear(in_features=1920, out_features=1920, bias=True)
        (rms_norm1): RMSNormBlock(
          (rmsNorm): RMSNorm((1920,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
        (rope): RotaryPositionalEmbedding()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1920, out_features=7680, bias=True)
        (swiglu): SwiGLU(
          (linear1): Linear(in_features=7680, out_features=3840, bias=True)
          (linear2): Linear(in_features=1920, out_features=7680, bias=True)
        )
        (ln2): Linear(in_features=7680, out_features=1920, bias=True)
      )
    

In [12]:
torch.autograd.set_detect_anomaly(True)  # debug amaçlı


In [13]:
model = model.train()

In [14]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(50000, 1920)
    )
  )
  (decoderBlocks): ModuleList(
    (0-29): 30 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1920, out_features=5760, bias=True)
        (o_proj): Linear(in_features=1920, out_features=1920, bias=True)
        (rms_norm1): RMSNormBlock(
          (rmsNorm): RMSNorm((1920,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
        (rope): RotaryPositionalEmbedding()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1920, out_features=7680, bias=True)
        (swiglu): SwiGLU(
          (linear1): Linear(in_features=7680, out_features=3840, bias=True)
          (linear2): Linear(in_features=1920, out_features=7680, bias=True)
        )
        (ln2): Linear(in_features=7680, out_features=1920, bias=True)
      )
    

Datasets Load

In [15]:
from datasets import Dataset

def keep_only_column(dataset: Dataset, keep_column: str) -> Dataset:
    """
    Bir Hugging Face dataset'inde sadece belirtilen sütunu tutar, diğer tüm sütunları kaldırır.

    Args:
        dataset (Dataset): Hugging Face Dataset nesnesi.
        keep_column (str): Korunacak sütunun adı.

    Returns:
        Dataset: Sadece seçilen sütunu içeren yeni dataset.
    """
    all_columns = dataset.column_names
    remove_columns = [col for col in all_columns if col != keep_column]
    return dataset.remove_columns(remove_columns)


In [16]:
from datasets import load_dataset, Dataset
from datasets import load_from_disk
import os

processed_path = "/media/hosman/Yedek/Datasets/CulturaY_3m"

if not os.path.exists(processed_path):
    datasetCulturaY = load_dataset("ontocord/CulturaY", "tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4)
    datasetCulturaY = datasetCulturaY.shuffle(seed=42).shuffle(seed=21).shuffle(seed=15).select(range(3_000_000))
    datasetCulturaY = datasetCulturaY.remove_columns(['id', 'document_lang',"scores","langs","url"])
    datasetCulturaY.save_to_disk(processed_path)
else:
    
    datasetCulturaY = load_from_disk("/media/hosman/Yedek/Datasets/CulturaY_3m").select(range(800_000))
    datasetCulturaY

Loading dataset from disk:   0%|          | 0/43 [00:00<?, ?it/s]

In [17]:
""" from datasets import load_dataset, Dataset
from datasets import load_from_disk
import os

processed_path = "/media/hosman/Yedek/Datasets/c4_tr_800k"

if not os.path.exists(processed_path):
    datasetC4 = load_dataset("allenai/c4", "tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4)
    datasetC4 = datasetC4.shuffle(seed=42)
    datasetC4 = datasetC4.select(range(800000)).remove_columns(['timestamp', 'url'])
    datasetC4.save_to_disk(processed_path)

else:
    datasetC4 = load_from_disk("/media/hosman/Yedek/Datasets/c4_tr_800k")
    datasetC4
 """

' from datasets import load_dataset, Dataset\nfrom datasets import load_from_disk\nimport os\n\nprocessed_path = "/media/hosman/Yedek/Datasets/c4_tr_800k"\n\nif not os.path.exists(processed_path):\n    datasetC4 = load_dataset("allenai/c4", "tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4)\n    datasetC4 = datasetC4.shuffle(seed=42)\n    datasetC4 = datasetC4.select(range(800000)).remove_columns([\'timestamp\', \'url\'])\n    datasetC4.save_to_disk(processed_path)\n\nelse:\n    datasetC4 = load_from_disk("/media/hosman/Yedek/Datasets/c4_tr_800k")\n    datasetC4\n '

In [18]:
from itertools import islice
from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk
from tqdm import tqdm
import os

processed_path = "/media/hosman/Yedek/Datasets/HPLT2.0_cleaned_3m"


if not os.path.exists(processed_path):
    print("Veri seti indiriliyor ve işleniyor...")
    
    cache_dir = "/media/hosman/Yedek/Datasets/"
    chunk_size = 100_000  # parça büyüklüğü
    total_size = 3_000_000

    streamed_dataset = load_dataset(
        "HPLT/HPLT2.0_cleaned",
        "tur_Latn",
        split="train",
        streaming=True,
        cache_dir=cache_dir
    )

    iterator = iter(streamed_dataset)
    all_chunks = []

    for i in tqdm(range(0, total_size, chunk_size), desc="Veriler alınıyor"):
        chunk = list(islice(iterator, chunk_size))
        if not chunk:
            break
        hf_chunk = Dataset.from_list(chunk)
        all_chunks.append(hf_chunk)

    # Tüm parçaları birleştir
    full_dataset = concatenate_datasets(all_chunks)
    full_dataset = keep_only_column(full_dataset , "text")
    # Disk'e kaydet
    full_dataset.save_to_disk(processed_path)
    print(f"{total_size} örnek başarıyla kaydedildi: {processed_path}")

else:
    print("Dataset daha önce kayıtlı:", processed_path)
    datasetHPLT2_cleaned_3m = load_from_disk(processed_path).shuffle(seed=42).shuffle(seed=21).shuffle(seed=15).select(range(800_000))
datasetHPLT2_cleaned_3m

Dataset daha önce kayıtlı: /media/hosman/Yedek/Datasets/HPLT2.0_cleaned_3m


Loading dataset from disk:   0%|          | 0/24 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 800000
})

In [19]:
datasetWiki = load_dataset("wikimedia/wikipedia", "20231101.tr",split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4).shuffle(seed=42).remove_columns(['id', 'url', 'title'])
datasetWiki

Dataset({
    features: ['text'],
    num_rows: 534988
})

In [20]:
""" datasetText = load_dataset("yasarefe/turkish-texts-dataset-2", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4).shuffle(seed=42)
datasetText """

' datasetText = load_dataset("yasarefe/turkish-texts-dataset-2", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4).shuffle(seed=42)\ndatasetText '

In [21]:
""" datasetOscarSmall = load_dataset("nthngdy/oscar-small", "unshuffled_deduplicated_tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", trust_remote_code=True, num_proc=4).shuffle(seed=42).select(range(300000)).shuffle(seed=42).select(range(300000)).remove_columns(['id'])
datasetOscarSmall """

' datasetOscarSmall = load_dataset("nthngdy/oscar-small", "unshuffled_deduplicated_tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", trust_remote_code=True, num_proc=4).shuffle(seed=42).select(range(300000)).shuffle(seed=42).select(range(300000)).remove_columns([\'id\'])\ndatasetOscarSmall '

In [22]:
from itertools import islice
from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk
from tqdm import tqdm
import os

processed_path = "/media/hosman/Yedek/Datasets/oscar_tr_1m"
cache_dir = "/media/hosman/Yedek/Datasets/"
chunk_size = 100_000  # parça büyüklüğü
total_size = 1_000_000

if not os.path.exists(processed_path):
    print("Veri seti indiriliyor ve işleniyor...")

    streamed_dataset = load_dataset(
        "oscar-corpus/OSCAR-2201",
        language="tr",
        split="train",
        streaming=True,
        cache_dir=cache_dir
    )

    iterator = iter(streamed_dataset)
    all_chunks = []

    for i in tqdm(range(0, total_size, chunk_size), desc="Veriler alınıyor"):
        chunk = list(islice(iterator, chunk_size))
        if not chunk:
            break
        hf_chunk = Dataset.from_list(chunk)
        all_chunks.append(hf_chunk)

    # Tüm parçaları birleştir
    full_dataset = concatenate_datasets(all_chunks)

    # Disk'e kaydet
    full_dataset.save_to_disk(processed_path)
    print(f"{total_size} örnek başarıyla kaydedildi: {processed_path}")

else:
    print("Dataset daha önce kayıtlı:", processed_path)
    datasetOscar = load_from_disk(processed_path).shuffle(seed=42).shuffle(seed=21).shuffle(seed=15).remove_columns(["id", "meta"])


Dataset daha önce kayıtlı: /media/hosman/Yedek/Datasets/oscar_tr_1m


In [23]:
dataset = concatenate_datasets([ datasetWiki, datasetOscar, datasetCulturaY, datasetHPLT2_cleaned_3m])

In [24]:
def replace_empty_with_none(example):
    # 'inputs' sütunundaki boş karakteri None ile değiştirelim
    if example['text'] == "":
        example['text'] = None
    return example

# dataset'teki 'inputs' sütunundaki boş karakterleri None ile değiştir

dataset = dataset.map(replace_empty_with_none)

In [25]:
dataset = dataset.filter(lambda x: x["text"]!=None)

In [26]:
import re

def clear_text(example):
    
    text = example["text"]

    text = re.sub(r'^[^a-zA-Z0-9çğıöşüÇĞİÖŞÜ]+', '', text)

    # Unicode boşluk karakterlerini normal boşluğa çevir
    text = re.sub(r'[\u2002\u2003\u2008\u2009\u200a\u202f\u2028\u3000\xa0]', ' ', text)
    # Garip karakterleri kaldır
    text = text.replace('\x85', '')
    # Normalleştir: fazla boşlukları sadeleştir
    text = re.sub(r'\s+', ' ', text).strip()

    example["text"] = text

    return example

dataset = dataset.map(clear_text)

In [27]:
dataset = dataset.filter(lambda x: "�" not in x["text"] and len(x["text"].strip()) > 0)

In [ ]:
dataset = dataset.filter(lambda x: (len(tokenizer.encode(x["text"]))) < max_seq_length)

Filter:   0%|          | 0/3128968 [00:00<?, ? examples/s]

In [ ]:
import unicodedata
import re

def is_latin(text):
    # Tüm karakterlerin Latin alfabesi veya basit semboller olup olmadığını kontrol eder
    for char in text:
        try:
            # Karakterin ait olduğu Unicode bloğunu al
            if "LATIN" not in unicodedata.name(char) and char.isalpha():
                return False
        except ValueError:
            # Karakterin Unicode adı yoksa (ör: emoji) → dışla
            return False
    return True

# Dataset'i filtrele
#dataset = dataset.filter(lambda x: is_latin(x["text"]))


In [ ]:
import re

# Sadece Latin harfleri ve bazı sembolleri içeren regex
latin_regex = re.compile(r"^[a-zA-ZçÇğĞıİöÖşŞüÜ0-9\s.,!?;:'\"()\[\]{}%€₺$@#&*°…—\-+/<>=~`^|\n\t]*$")

def is_latin_simple(text):
    return bool(latin_regex.match(text))

dataset = dataset.filter(lambda x: is_latin_simple(x["text"]))


In [ ]:
""" karakter_seti = set()

for example in tqdm(dataset):
    text = example["text"]
    if text is not None:
        karakter_seti.update(text)

print(karakter_seti) """

In [ ]:
dataset

In [ ]:
print(dataset[501]["text"])

In [ ]:
print(tokenizer.tokenize(dataset[5]["text"]))

In [ ]:
print(tokenizer.decode(tokenizer.encode(dataset[501]["text"])))

In [ ]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.shuffle(seed=41)
dataset = dataset.shuffle(seed=40)
dataset = dataset.shuffle(seed=39).select(range(300_000))

Dataset Save

In [ ]:
with open("Tokenizer/BPE_TokenizerTexts.txt", "w", encoding="utf-8") as f:
    for example in tqdm(dataset):
        text = example["text"].strip()
        if text:  # Boş satırları atla
            f.write(text + "\n")
 

Kişisel Tokenizer Ayarlama

In [ ]:
text = "Bugün 3 arkadaş saat 14:45’te Kadıköy’e gitti; kahve içip Python çalıştılar! 😊"
tokens = tokenizer.tokenize(text)
ids = tokenizer.encode(text)

print("🔤 Tokens:", tokens)
print("🔢 Token IDs:", ids)
print("📜 Çözümlenmiş:", tokenizer.decode(ids))

Kişisel Tokenizer Bölümü Bitişi

In [ ]:
from datasets import DatasetDict

# 1. Veriyi train ve test olarak ayırma
# Örneğin, dataset zaten tek bir büyük veri seti (örneğin "data") içeriyor
# Bunu %80 train ve %20 test olarak bölelim
train_dataset, temp_dataset = dataset.train_test_split(test_size=0.1, seed=42).values()

# 2. Test setini de %50 validation ve %50 test olarak bölelim
val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.2, seed=42).values()

In [ ]:
#model.gradient_checkpointing_enable()
model.use_cache = True

In [ ]:
import wandb

wb_c = wandb.init(project="Basic LLM Train", name="./Crispy-2.8B-CLM" , resume="allow", id="l6l7x3ob") #id="a7zeymst",id="ecibz7e4" id="dbaxrwf4"
wb_c.watch(model, log="all")

In [ ]:
import wandb
import evaluate
import numpy as np
from prettytable import PrettyTable
import torch
import re
from rapidfuzz import fuzz

def exact_match(prediction, reference):
    return prediction.strip().lower() == reference.strip().lower()

def contains_correct_result(prediction, reference):
    try:
        ref_nums = [int(s) for s in re.findall(r"\d+", reference)]
        pred_nums = [int(s) for s in re.findall(r"\d+", prediction)]
        return any(num in pred_nums for num in ref_nums)
    except:
        return False

def fuzzy_match_score(prediction, reference):
    return fuzz.ratio(prediction, reference) / 100.0  # normalize to 0-1

def evaluate_model(model, tokenizer, test_dataset, max_seq_length=256):
    """
    Eğitilmiş modeli test veri kümesi üzerinde değerlendirir ve sonuçları wandb'a loglar.
    
    Parametreler:
    - model: Eğitilmiş dil modeli
    - tokenizer: Modelin tokenizer'ı
    - test_dataset: Test veri kümesi (instruction-output içermeli)
    - max_seq_length: Maksimum yanıt uzunluğu (varsayılan: 256)

    Çıktı:
    - Metin tablosu (PrettyTable ile)
    - wandb logları
    """

    # Değerlendirme metriklerini yükleme
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    meteor = evaluate.load("meteor")
    bertscore = evaluate.load("bertscore")

    predictions = []
    references = []
    exact_matches = []
    correct_results = []
    fuzzy_scores = []

    # Modeli değerlendirme moduna al
    model.eval()

    print("🚀 Model test verisi üzerinde değerlendiriliyor...\n")

    for example in test_dataset:
        input_text = f"### Talimat:\n{example['instruction']}\n\n### Yanıt:\n"
        reference_text = example["output"]

        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)

        decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        decoded_output = decoded_output.split("### Yanıt")[-1].strip()

        predictions.append(decoded_output)
        references.append(reference_text)

        exact_matches.append(exact_match(decoded_output, reference_text))
        correct_results.append(contains_correct_result(decoded_output, reference_text))
        fuzzy_scores.append(fuzzy_match_score(decoded_output, reference_text))

    # Metrik hesaplamaları
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    meteor_score = meteor.compute(predictions=predictions, references=references)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="tr")

    bert_precision = np.mean(bert_scores["precision"])
    bert_recall = np.mean(bert_scores["recall"])
    bert_f1 = np.mean(bert_scores["f1"])
    exact_match_score = np.mean(exact_matches)
    correct_result_score = np.mean(correct_results)
    fuzzy_match_avg = np.mean(fuzzy_scores)

    # Sonuçları tabloya ekle
    table = PrettyTable()
    table.field_names = ["Metrik", "Değer"]
    table.add_row(["ROUGE-1", round(rouge_scores["rouge1"], 4)])
    table.add_row(["ROUGE-2", round(rouge_scores["rouge2"], 4)])
    table.add_row(["ROUGE-L", round(rouge_scores["rougeL"], 4)])
    table.add_row(["BLEU", round(bleu_score["bleu"], 4)])
    table.add_row(["METEOR", round(meteor_score["meteor"], 4)])
    table.add_row(["BERTScore Precision", round(bert_precision, 4)])
    table.add_row(["BERTScore Recall", round(bert_recall, 4)])
    table.add_row(["BERTScore F1", round(bert_f1, 4)])
    table.add_row(["Exact Match", round(exact_match_score, 4)])
    table.add_row(["Contains Correct Result", round(correct_result_score, 4)])
    table.add_row(["Fuzzy Match", round(fuzzy_match_avg, 4)])

    # Sonuçları yazdır
    print(table)

    # wandb log
    wandb.log({
        "ROUGE-1": rouge_scores["rouge1"],
        "ROUGE-2": rouge_scores["rouge2"],
        "ROUGE-L": rouge_scores["rougeL"],
        "BLEU": bleu_score["bleu"],
        "METEOR": meteor_score["meteor"],
        "BERTScore Precision": bert_precision,
        "BERTScore Recall": bert_recall,
        "BERTScore F1": bert_f1,
        "Exact Match": exact_match_score,
        "Contains Correct Result": correct_result_score,
        "Fuzzy Match": fuzzy_match_avg
    })

    print("\n✅ Model değerlendirme tamamlandı ve tüm metrikler wandb'a loglandı.")


In [ ]:
train_dataset[0]["text"]

In [ ]:
#model.gradient_checkpointing_enable()

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
#val_dataset = val_dataset.select(range(10100, 11000))

In [ ]:
import math

def get_warmup_steps_from_dataset(dataset_len, batch_size, num_epochs, pct=0.05):
    """
    Dataset bilgisine göre dinamik warmup step sayısı hesaplar.

    Args:
        dataset_len (int): Dataset’teki toplam örnek sayısı.
        batch_size (int): Batch başına örnek sayısı.
        num_epochs (int): Toplam epoch sayısı.
        pct (float): Warmup oranı (0.03 - 0.1 arası önerilir).

    Returns:
        int: Warmup step sayısı.
    """
    steps_per_epoch = math.ceil(dataset_len / batch_size)
    total_steps = steps_per_epoch * num_epochs
    warmup_steps = int(total_steps * pct)
    return warmup_steps


In [ ]:
from transformers import TrainerCallback
import torch
import wandb
import os
import shutil

class GradientCheckCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        model = kwargs["model"]

        found_problem = False
        for name, param in model.named_parameters():
            if param.grad is not None:
                if torch.isnan(param.grad).any():
                    print(f"🚨 NaN in gradients of {name}")
                    found_problem = True
                if torch.isinf(param.grad).any():
                    print(f"🚨 Inf in gradients of {name}")
                    found_problem = True

        if found_problem:
            print(f"⛔ Problematic gradients detected at step {state.global_step}!")
            
            control.should_training_stop = True  # Eğitimi durdur


        return control


class ManualGradientClipCallback(TrainerCallback):
    def __init__(self, max_grad_norm=1.0):
        self.max_grad_norm = max_grad_norm

    def on_step_end(self, args, state, control, **kwargs):
        model = kwargs["model"]

        # Gradyanları kliple
        total_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), self.max_grad_norm
        )

        if torch.isnan(total_norm) or torch.isinf(total_norm):
            print(f"🚨 NaN/Inf gradyan normu! Step: {state.global_step}")
        elif total_norm > self.max_grad_norm:
            print(f"⚠️ Gradyan norm ({total_norm:.2f}) sınırı aştı, kliplendi.")

        return control


class WandbTextGenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, log_interval=50, device="cuda"):
        self.tokenizer = tokenizer
        self.prompts = [
                                "Ali sabah uyanır ve pencereden dışarı bakar. Hava",
                                "Küçük kız elindeki balonla parka doğru yürürken",
                                "Üniversite sınav sonuçları açıklandığında",
                                "Yağmurlu bir günde eski kitapçıda",
                                "Gece boyunca ormanda duyulan garip sesler",
                                "Deniz kenarında yürüyen yaşlı adamın aklında",
                                "Robotlar gelecekte insanların işlerini",
                                "İstanbul'un kalabalık sokaklarında bir adam",
                                "Yaz tatilinde köye giden çocuklar",
                                "Bir sabah, dünya üzerindeki tüm elektrik",
                                "Sakin bir kasabada geçen sır dolu bir hikaye",
                                "Sabah kahvemi içerken aklımdan geçen tek şey",
                                "Karanlık sokakta ilerlerken aniden",
                                "Uzay gemisi bilinmeyen bir gezegene indiğinde",
                                "Büyükannemin anlattığı eski zaman hikayeleri",
                                "Dün gece gördüğüm rüya hâlâ aklımda",
                                "Sınıfta öğretmenin sorduğu zor soru karşısında",
                                "Bir zamanlar uzak bir ülkede yaşayan bir kral",
                                "Kütüphanenin en köşesinde tozlu bir kitap",
                                "Gözlerini açtığında bambaşka bir dünyadaydı"
                            ]

        self.log_interval = log_interval
        self.device = device
        


    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.log_interval == 0 and state.global_step != 0:
            model = kwargs['model'].to(self.device)
            self.table = wandb.Table(columns=["prompt_number", "step", "prompt", "output"])

            for i, prompt in enumerate(self.prompts):
                # Tokenize prompt and move to correct device + dtype
                inputs = self.tokenizer(prompt,padding="max_length",  max_length=max_seq_length, return_tensors="pt").to(self.device)
                #input_ids = input_ids.to(dtype=model_dtype)

                #max_new_tokens = max_seq_length - inputs["input_ids"].shape[1]
                max_new_tokens = 100
                with torch.no_grad():
                    generated_ids = model.generate(
                                                **inputs, 
                                                max_new_tokens=max_new_tokens, 
                                                use_cache=True, 
                                                do_sample=True,
                                                top_k=50,                          # En iyi 50 token içinden seç
                                                top_p=0.95,                        # Kümülatif olasılığı %95'e kadar olanlardan seç
                                                repetition_penalty=1.2,  
                                                pad_token_id=tokenizer.pad_token_id,
                                                eos_token_id=tokenizer.eos_token_id,
                                                bos_token_id=tokenizer.bos_token_id  # Eklenebilir
                                                )
                output_text = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

                # Tabloya ekle
                self.table.add_data(i, state.global_step, prompt, output_text)

            # WandB'ye logla
            wandb.log({"text_generation/table": self.table}, step=state.global_step)
            
            #print(f"\n🧪 [Step {state.global_step}] Prompt Testi:\n🟢 Prompt: {prompt}\n🔵 Output: {output_text}")


class WandbModelSaverCallback(TrainerCallback):
    def __init__(self, save_interval=500):
        self.artifacts = []
        self.save_interval = save_interval

    def on_save(self, args, state, control, **kwargs):
        if state.global_step % self.save_interval != 0:
            return control  # ⛔ Save interval dışında, hiçbir şey yapma

        checkpoint_dir = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        artifact_name = f"crispy-checkpoint-{state.global_step}"
        artifact = wandb.Artifact(name=artifact_name, type="model")

        artifact.add_dir(checkpoint_dir)
        wandb.log_artifact(artifact)
        self.artifacts.append(artifact_name)

        # 🧹 Temizlik: WandB staging cache
        staging_dir = os.path.join(os.path.expanduser("~"), ".local", "share", "wandb", "artifacts", "staging")
        try:
            shutil.rmtree(staging_dir)
            print(f"✅ Cleaned WandB staging folder: {staging_dir}")
        except Exception as e:
            print(f"⚠️ Could not clean staging folder: {e}")

        return control


In [ ]:
train_dataset[345]

In [ ]:
def shift_labels(example):
    input_ids = example["input_ids"]
    labels = input_ids.copy()
    labels[:-1] = input_ids[1:]
    labels[-1] = tokenizer.pad_token_id
    example["labels"] = labels
    return example



def tokenize_fn(example):
   
    full_text = example["text"]
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )
    
    tokenized["input_ids"] = tokenized["input_ids"][0]
    tokenized["labels"] = tokenized["input_ids"].clone()
    tokenized["attention_mask"] = tokenized["attention_mask"][0]
    

    return tokenized 

#train_dataset = train_dataset.map(tokenize_fn, remove_columns=train_dataset.column_names)
#val_dataset = val_dataset.map(tokenize_fn, remove_columns=val_dataset.column_names)
#test_dataset = test_dataset.map(tokenize_fn, remove_columns=test_dataset.column_names)

from tqdm import tqdm
from datasets import Dataset
import torch
from torch.nn.utils.rnn import pad_sequence

def tokenize_and_chunk(dataset, tokenizer, max_length, add_bos_eos=True, desc="Processing"):
    all_tokens = []

    print(f"🔄 Tokenizing {len(dataset)} examples...")

    for example in tqdm(dataset, desc=f"{desc} → Tokenizing"):
        tokens = tokenizer(
            example["text"],
            add_special_tokens=False,
            return_attention_mask=False,
            return_token_type_ids=False
        )["input_ids"]
        all_tokens.extend(tokens)

    print(f"🔗 Total tokens: {len(all_tokens)}")
    print(f"✂️ Chunking with max_length={max_length}...")

    chunks = []
    chunk_step = max_length - 2 if add_bos_eos else max_length
    bos_token_id = tokenizer.bos_token_id or tokenizer.cls_token_id or 0
    eos_token_id = tokenizer.eos_token_id or tokenizer.sep_token_id or 2
    pad_token_id = tokenizer.pad_token_id or 0

    for i in tqdm(range(0, len(all_tokens), chunk_step), desc=f"{desc} → Chunking"):
        chunk = all_tokens[i : i + chunk_step]
        if len(chunk) < 32:  # çok kısa chunk'ları atla (isteğe bağlı)
            continue

        if add_bos_eos:
            chunk = [bos_token_id] + chunk + [eos_token_id]

        # Pad ile 2048'e sabitle
        if len(chunk) < max_length:
            chunk += [pad_token_id] * (max_length - len(chunk))

        chunks.append({"input_ids": chunk})

    print(f"✅ Created {len(chunks)} padded chunks (length={max_length}).")
    return Dataset.from_list(chunks)




from datasets import Dataset

train_dataset = tokenize_and_chunk(train_dataset, tokenizer, max_length=max_seq_length)

val_dataset = tokenize_and_chunk(val_dataset, tokenizer, max_length=max_seq_length)

test_dataset = tokenize_and_chunk(test_dataset, tokenizer, max_length=max_seq_length)


In [ ]:
from collections import Counter

def analyze_token_distribution(dataset, tokenizer, sample_size=20000):
    unk_id = tokenizer.unk_token_id
    pad_id = tokenizer.pad_token_id
    
    unk_count = 0
    pad_count = 0
    total_tokens = 0

    # Sınırlı sayıda örnek ile analiz (gerekirse tamamında yapılabilir)
    for i in range(min(sample_size, len(dataset))):
        ids = dataset[i]["input_ids"]
        unk_count += sum(1 for token in ids if token == unk_id)
        pad_count += sum(1 for token in ids if token == pad_id)
        total_tokens += len(ids)
    
    unk_ratio = unk_count / total_tokens
    pad_ratio = pad_count / total_tokens

    print(f"🔎 İncelenen örnek sayısı: {min(sample_size, len(dataset))}")
    print(f"📦 Toplam token sayısı: {total_tokens}")
    print(f"❓ UNK token oranı: {unk_ratio:.4%}")
    print(f"🔲 PAD token oranı: {pad_ratio:.4%}")

print("🔍 TRAIN SET")
analyze_token_distribution(train_dataset, tokenizer)

print("\n🔍 VALIDATION SET")
analyze_token_distribution(val_dataset, tokenizer)

print("\n🔍 TEST SET")
analyze_token_distribution(test_dataset, tokenizer)

In [ ]:
# İlk 3 örneği çöz
for i in range(5):
    print(f"🔹 Chunk {i+1}:")
    print(tokenizer.decode(train_dataset[i]["input_ids"], skip_special_tokens=True))


In [ ]:
num_epochs = 3

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class CustomCausalLMDataCollator:
    tokenizer: Any
    pad_token_id: int = None

    def __post_init__(self):
        if self.pad_token_id is None:
            self.pad_token_id = self.tokenizer.pad_token_id or 0

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_ids = [torch.tensor(f["input_ids"], dtype=torch.long) for f in features]

        # Pad sequence'ler
        input_ids_padded = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.pad_token_id
        )

        # Attention mask: pad_token != 0 olan yerlere 1
        attention_mask = (input_ids_padded != self.pad_token_id).long()

        # Label: input_ids'in doğrudan kopyası (shift modelde yapılacak)
        labels = input_ids_padded.clone()

        return {
            "input_ids": input_ids_padded,
            "attention_mask": attention_mask,
            "labels": labels
        }

collator = CustomCausalLMDataCollator(tokenizer)

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

args = TrainingArguments(
    # 🚀 Eğitim Temelleri
    num_train_epochs=num_epochs,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=64,
    eval_accumulation_steps=64,
    output_dir="./Crispy-2.8B-CLM",
    seed=3407,
    no_cuda=False,
    use_cpu=False,
    auto_find_batch_size=False,

    # 🧠 Optimizasyon
    optim="adamw_torch_fused",
    learning_rate=0.01,
    weight_decay=0.01,
    adam_beta2=0.95,
    max_grad_norm=1.0,

    # 🌀 Öğrenme Oranı Planlayıcı
    lr_scheduler_type="cosine",
    #lr_scheduler_kwargs={"num_cycles": 5},
    warmup_ratio= 0.05*2,  # num_epochs = 2 ise

    # 🔄 Değerlendirme & Checkpoint
    eval_strategy="steps",
    eval_steps=1000,
    save_steps=50,
    save_total_limit=2,

    # 🧠 Precision Ayarları
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),

    # 📜 Loglama & İzleme
    logging_strategy="steps",
    logging_steps=50,
    #log_level="debug",              # Ana işlem log seviyesi
    #log_level_replica="warning",    # Diğer işlem log seviyesi (dağıtık eğitimde)
    report_to=["wandb"],
    logging_nan_inf_filter=True,

    # 🧹 Bellek ve Checkpointing
    gradient_checkpointing=False,
    torch_empty_cache_steps=50,
)


In [ ]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq, Trainer

trainer = Trainer(
    model = model,
    #tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator=collator,
    #dataset_text_field = "text",
    #max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    #dataset_num_proc = 2,
    #packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    #packing=False,
    #remove_unused_columns=True,
    #torch_compile=True,
    callbacks=[WandbTextGenerationCallback(tokenizer=tokenizer, log_interval=50), 
               GradientCheckCallback(), 
               ManualGradientClipCallback(), 
               #WandbModelSaverCallback(save_interval=250) 
               ],
    args = args
)

In [ ]:
trainer.train(
                resume_from_checkpoint=False
            )

In [ ]:
# Test değerlendirmesi
#evaluate_model(model, tokenizer, test_dataset, max_seq_length=max_seq_length)

In [ ]:
# 6. Eğitilmiş Modeli Kaydedin
model.save_pretrained("./Crispy-2.8B-CLM")
tokenizer.save_pretrained("./Crispy-2.8B-CLM")

print("Eğitim tamamlandı ve model kaydedildi.")

In [ ]:
import unsloth
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import torch
from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

In [ ]:
from transformers import XLMRobertaTokenizer

# XLM-Roberta tokenizer yükleniyor
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
special_tokens_dict = {
    "bos_token": "<s>",
    "eos_token": "<|eot_id|>",
    "additional_special_tokens":  [
        "<|im_start|>", "<|im_end|>",
        "<|system|>", "<|user|>", "<|assistant|>",
        "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"
    ]
}


tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
import torch

# Model ve tokenizer'ını yükle
from transformers import PreTrainedTokenizerFast
from transformers import AutoConfig, AutoModelForCausalLM
from MyLLM.CrispyLLM_RoPE2.modeling_crispy_rope import CrispyLLMConfig, CrispyForCausalLM


# 3. Kayıt (Auto ile kullanabilmek için)
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)
model = AutoModelForCausalLM.from_pretrained("./Crispy-330M-V3-Rope-NewTokenizer-JustLanguage/checkpoint-8500" ,  
                                            attn_implementation="flash_attention_2",
                                            trust_remote_code=True,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto"
      ).cuda().eval()


In [ ]:
""" 
# Sohbet geçmişi
chat_history = ""

# Cevap üretme fonksiyonu
def generate_response(prompt, max_new_tokens=256):
    input_text = chat_history + prompt
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = output_text[len(input_text):].strip()
    return response

print("🧠 Crispy Chatbot hazır! Çıkmak için Ctrl+C, sıfırlamak için '/reset' yaz.")
print("-" * 50)

# Sonsuz konuşma döngüsü
while True:
    user_input = input("👤 Sen: ")
    
    if user_input.strip().lower() == "/reset":
        chat_history = ""
        print("🔁 Sohbet sıfırlandı.")
        continue

    chat_history += f"👤 Sen: {user_input}\n"
    response = generate_response(f"👤 Sen: {user_input}\n🤖 Crispy:")
    chat_history += f"🤖 Crispy: {response}\n"

    print(f"🤖 Crispy: {response}")
 """

In [ ]:
input_text = """Ali sabah uyanır ve pencereden dışarı bakar. Hava"""

In [ ]:
input_ids  = tokenizer(input_text, padding="max_length", max_length=1024,return_tensors="pt").to(model.device)

with torch.no_grad():
    # Modelden yanıt üret
    generated_ids = model.generate(
        **input_ids, 
        max_new_tokens=1024 ,
        do_sample=False,
        use_cache=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        #num_beams=5, 
        no_repeat_ngram_size=3,  
        early_stopping=True,
       # top_k=50,
       # top_p=0.9,
        #temperature=0.9,
    )

# Üretilen token'ları geri metne çevir
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text[len(input_text):])

